In [1]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import spei as si

In [2]:
df = pd.read_csv("../data/climate_data.csv", parse_dates=["date"])

In [3]:
df.head()

,province,date,variable,value
0,Araba/Álava,2000-01-01,avg_precipitation,22.252368
1,Araba/Álava,2000-02-01,avg_precipitation,23.672784
2,Araba/Álava,2000-03-01,avg_precipitation,50.380843
3,Araba/Álava,2000-04-01,avg_precipitation,122.973462
4,Araba/Álava,2000-05-01,avg_precipitation,69.824049


In [4]:
df.dtypes

province            object
date        datetime64[ns]
variable            object
value              float64
dtype: object

In [5]:
df = (
    pd.pivot(data=df, index=["province", "date"], columns=["variable"], values="value")
    .reset_index()
    .rename_axis(columns="")
)

In [6]:
df.head()

,province,date,avg_pet,avg_precipitation
0,A Coruña,2000-01-01,262.324570,57.015631
1,A Coruña,2000-02-01,375.382717,59.520040
2,A Coruña,2000-03-01,680.471894,48.476642
3,A Coruña,2000-04-01,631.914884,225.902888
4,A Coruña,2000-05-01,971.068998,95.572749


In [7]:
df.dtypes


province                     object
date                 datetime64[ns]
avg_pet                     float64
avg_precipitation           float64
dtype: object

In [8]:
df["water_balance"] = df["avg_precipitation"] - df["avg_pet"]

In [9]:
df.head()

,province,date,avg_pet,avg_precipitation,water_balance
0,A Coruña,2000-01-01,262.324570,57.015631,-205.308939
1,A Coruña,2000-02-01,375.382717,59.520040,-315.862677
2,A Coruña,2000-03-01,680.471894,48.476642,-631.995251
3,A Coruña,2000-04-01,631.914884,225.902888,-406.011996
4,A Coruña,2000-05-01,971.068998,95.572749,-875.496249


In [10]:
spei_df = (
    df.pivot(index="date", columns="province", values="water_balance")
    .apply(lambda x: si.spei(x, timescale=1), axis=0)
    .unstack()
    .reset_index(name="spei")
)

In [11]:
spei_df.head()

,province,date,spei
0,A Coruña,2000-01-01,-0.453178
1,A Coruña,2000-02-01,-0.305103
2,A Coruña,2000-03-01,-1.100348
3,A Coruña,2000-04-01,2.312600
4,A Coruña,2000-05-01,1.032068


In [12]:
spei_df.head()

,province,date,spei
0,A Coruña,2000-01-01,-0.453178
1,A Coruña,2000-02-01,-0.305103
2,A Coruña,2000-03-01,-1.100348
3,A Coruña,2000-04-01,2.312600
4,A Coruña,2000-05-01,1.032068


In [13]:
df.head()

,province,date,avg_pet,avg_precipitation,water_balance
0,A Coruña,2000-01-01,262.324570,57.015631,-205.308939
1,A Coruña,2000-02-01,375.382717,59.520040,-315.862677
2,A Coruña,2000-03-01,680.471894,48.476642,-631.995251
3,A Coruña,2000-04-01,631.914884,225.902888,-406.011996
4,A Coruña,2000-05-01,971.068998,95.572749,-875.496249


In [14]:
df = pd.merge(left=df, right=spei_df, on=["date", "province"], how="inner")

In [15]:
df.head()

,province,date,avg_pet,avg_precipitation,water_balance,spei
0,A Coruña,2000-01-01,262.324570,57.015631,-205.308939,-0.453178
1,A Coruña,2000-02-01,375.382717,59.520040,-315.862677,-0.305103
2,A Coruña,2000-03-01,680.471894,48.476642,-631.995251,-1.100348
3,A Coruña,2000-04-01,631.914884,225.902888,-406.011996,2.312600
4,A Coruña,2000-05-01,971.068998,95.572749,-875.496249,1.032068


In [16]:
df.to_csv('../data/spei.csv', index=False)